## Initialize and import all the required libraries

In [1]:
import io
import requests
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from io import StringIO
import pandas as pd
import numpy as np
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe   
import re
pd.set_option('display.max_rows', None)

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
!pip install geopandas
!pip install pygeocoder
!pip install geocoder

     |████████████████████████████████| 962 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 38.3 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 1.0 MB/s eta 0:00:01
  Created wheel for pygeocoder: filename=pygeocoder-1.2.5-py3-none-any.whl size=8883 sha256=7f7ca08cd3c0021c2211b0f244ed03117e548ffb6e2cd975b13f17640dbac864
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/fd/77/97/9c0c6cd57eed0daa759d2ad98217602333225936236862528c
Successfully built pygeocoder
     |████████████████████████████████| 98 kB 7.5 MB/s  eta 0:00:01


In [4]:
pip install pgeocode

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

## Read the Property sales csv to map zip code

In [7]:
housing_df=pd.read_csv('https://s3.amazonaws.com/vbgov-ckan-open-data/Property_Sales.csv',skipinitialspace=True)
housing_df.head()
housing_df['Zip Code'].replace(r'\t', np.nan,inplace=True,regex=True)

In [8]:
housing_df = housing_df[housing_df['Zip Code'].notna()]
housing_df.head()
housing_df.reset_index(drop=True,inplace=True)
housing_df.head()

,GPIN,Street Address,City,State,Zip Code,Neighborhood,Land Value,Improvement Value,Total Value,Land USE (Y/N),Sale Date,Sale Price,Document Number,Deed Book,Deed Page
0,13871638150000,3536 Baum Rd,Virginia Beach,VA,23457-1017,Blackwater,211900,158700,370600,No,04/04/1995,0,000000000000000,3486,1097.0
1,13871638150000,3536 Baum Rd,Virginia Beach,VA,23457-1017,Blackwater,211900,158700,370600,No,08/24/2007,310000,20070824001158500,NaN,NaN
2,13875868830000,3301 Indian Creek Rd,Virginia Beach,VA,23457,Blackwater Land Use,334700,346100,680800,Yes,08/15/2014,160000,20140815000762580,NaN,NaN
3,13875868830000,3301 Indian Creek Rd,Virginia Beach,VA,23457,Blackwater Land Use,334700,346100,680800,Yes,11/19/2015,230000,20151119001129190,NaN,NaN
4,13876818850000,3285 Indian Creek Rd,Virginia Beach,VA,23457,Blackwater Land Use,191400,344600,536000,Yes,07/10/2014,153000,20140711000628700,NaN,NaN


## Extract Zip code and Neighborhood from property sales CSV
## Also the count per zip code per neighborhood is also taken to obtain the zip code that occurs most for a neighborhood

In [9]:
##print(housing_df.shape)
##housing_df.groupby('Neighborhood')['Zip Code']##.to_frame().reset_index().sort_values(by='Neighborhood')
Zip_df=housing_df[['Zip Code', 'Neighborhood']] 
Zip_df = Zip_df.loc[Zip_df['Zip Code']!="\t"]
pd.set_option('display.max_rows', 10)
##Zip_df['Neighborhood'].nunique()
##Zip_df.reset_index(inplace=True)
Zip_df['Zip Code']=Zip_df['Zip Code'].str.slice(0, 5)
z1=Zip_df.groupby(['Neighborhood','Zip Code']).size().to_frame('count')
z1
idx=z1.groupby(['Neighborhood'])['count'].idxmax()
Zip_df=z1.loc[idx,].reset_index()
Zip_df['count'] = Zip_df['count'].astype(int)
Zip_df

,Neighborhood,Zip Code,count
0,3556 Shore Drive Condo,23455,141
1,3800 Oceanfront Condo,23451,185
2,921 Oceans Condo,23451,60
3,Acreage,23464,53
4,Acredale,23464,863
...,...,...,...
916,Woodstock Point Condo,23464,59
917,Wordsworth Vill/Browningstone,23456,1510
918,Wynd Crest Condo,23456,437
919,Wyndamere,23456,676


In [10]:
Zip_df=Zip_df.drop_duplicates()
Zip_df.reset_index(drop=True,inplace=True)

# Initialize the latitiude and longitude to VA Beach to plot the map later

In [11]:
geolocator = Nominatim(user_agent="VB_explorer")
##location = geolocator.geocode(address)
location = geolocator.geocode(query={'postalcode':'23464'}, addressdetails=True)
print(location)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of VB are {}, {}.'.format(latitude, longitude))

Virginia Beach, Virginia, 23464, United States
The geograpical coordinate of VB are 36.79809364440139, -76.17643670340314.


In [12]:
import pgeocode
nomi = pgeocode.Nominatim('us')

## Read the Crime data text file

In [13]:
# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_d2f4bf54b8264056917ceb6ff69c6ef1.get_object(Bucket='courseracapstoneproject-donotdelete-pr-t3xucvjcfuedgv', Key='Crime_200912lozn2rxkyehda0fs22dhovon.txt')['Body']
# add missing __iter__ method so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 
crime_df = pd.read_csv(io.StringIO(streaming_body_2.read().decode('utf-8')),sep='\t', header=None)
##csv_string = streaming_body_2.read().decode('utf-8')   
##print(csv_string)
##crime_df = pd.read_csv(csv_string,sep='\t', header=None)
crime_df[0]
df1 = crime_df[0].str[:60]
df2=crime_df[0].str[60:135]
df3=crime_df[0].str[135:148]
print(type(df1))
crime_df_final = pd.DataFrame({'Neighborhood': df1, 'Crime Description': df2,'Crime Count':df3})
crime_df_final.drop([0 , 20011], inplace=True)
crime_df_final.reset_index
crime_df_final['Crime Count'] = crime_df_final['Crime Count'].astype(int)
crime_df_final

<class 'pandas.core.series.Series'>


,Neighborhood,Crime Description,Crime Count
1,[Undetermined] ...,ABDUCTION/ KIDNAPPING (100A) ...,7
2,[Undetermined] ...,ALL OTHER REPORTABLE OFFENSES (90ZZ) ...,28
3,[Undetermined] ...,ANNOYING PHONE CALLS/ HARASSING COMMUNICATION ...,15
4,[Undetermined] ...,ARSON (200 ) ...,12
5,[Undetermined] ...,"ASSAULT, AGGRAVATED (13A1) ...",26
...,...,...,...
20006,ZAYRES PLAZA SHPG CTR IN WITCHDUCK ...,"FRAUD, IDENTITY THEFT (26A7) ...",1
20007,ZAYRES PLAZA SHPG CTR IN WITCHDUCK ...,"FRAUD, TRICK, FALSE PRETENSES (26A3) ...",1
20008,ZAYRES PLAZA SHPG CTR IN WITCHDUCK ...,HIT & RUN (90ZC) ...,1
20009,ZAYRES PLAZA SHPG CTR IN WITCHDUCK ...,"LARCENY, FROM MOTOR VEHICLE (23F ) ...",3


## Group by Neighborhood to get the sum of crime incident counts

In [14]:
crime_df_summary = crime_df_final.groupby('Neighborhood')['Crime Count'].sum().to_frame().reset_index()
##crime_df_summary.set_index('Neighborhood',inplace= True)
crime_df_summary['Neighborhood'] = crime_df_summary['Neighborhood'].str.strip()
crime_df_summary['Neighborhood'] = crime_df_summary['Neighborhood'].str.title()
print(crime_df_summary.columns)
crime_df_summary

Index(['Neighborhood', 'Crime Count'], dtype='object')


,Neighborhood,Crime Count
0,Abingdon Village In College Park,189
1,Acredale,271
2,Adam`S Glen In Glenwood,4
3,Adkins Reserve Condos,18
4,Adult Learning Center School,1
...,...,...
1142,Woodstock Point,5
1143,Wynd Crest Condos,25
1144,Wyndamere In Salem Lakes,45
1145,Zayres Plaza Shpg Ctr In Witchduck,8


## Read the Zillow Home vaue index data

In [15]:
body = client_d2f4bf54b8264056917ceb6ff69c6ef1.get_object(Bucket='courseracapstoneproject-donotdelete-pr-t3xucvjcfuedgv',Key='Neighborhood_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon_Nov.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

Home_value_zillow = pd.read_csv(body)
Home_value_zillow.head()


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30
0,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,132343.0,...,324845.0,324917.0,326237.0,328195.0,330590.0,333233.0,335638.0,338601.0,343524.0,348957.0
1,112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,NaN,...,190385.0,192596.0,195149.0,197695.0,200059.0,202769.0,205996.0,209917.0,213816.0,218411.0
2,192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,138996.0,...,268916.0,270899.0,272631.0,273723.0,274122.0,275312.0,277788.0,281152.0,283765.0,286100.0
3,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,254412.0,...,1254324.0,1243722.0,1244017.0,1243078.0,1241951.0,1247438.0,1252656.0,1258274.0,1252099.0,1245560.0
4,118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,133918.0,...,529049.0,534504.0,539172.0,541698.0,544530.0,549764.0,557257.0,565000.0,571646.0,577043.0


## Extract only the latest HVI value

In [16]:
Home_value_zillow_vb=Home_value_zillow.loc[Home_value_zillow.City =='Virginia Beach'][['RegionID','City','RegionName','2020-11-30']]
Home_value_zillow_vb = Home_value_zillow_vb.rename({'RegionName': 'Neighborhood_zillow','2020-11-30':'Median_HVI'}, axis=1)
Home_value_zillow_vb.reset_index(drop=True, inplace=True)
print(Home_value_zillow_vb.columns)
Home_value_zillow_vb

Index(['RegionID', 'City', 'Neighborhood_zillow', 'Median_HVI'], dtype='object')


,RegionID,City,Neighborhood_zillow,Median_HVI
0,246678,Virginia Beach,Princess Anne,369781.0
1,244707,Virginia Beach,Kempsville,256486.0
2,247145,Virginia Beach,Salem,299268.0
3,241514,Virginia Beach,Bayside,253829.0
4,245508,Virginia Beach,Mears Corner,292956.0
...,...,...,...,...
26,417477,Virginia Beach,First Landing State Park,1103866.0
27,418158,Virginia Beach,Croatan Beach,870809.0
28,242226,Virginia Beach,Cape Story by the Sea,696546.0
29,247090,Virginia Beach,Rudee Heights,380966.0


## Install Fuzzy logic compare package

In [17]:
!pip install fuzzywuzzy

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Using fuzzy logic compare neighborhood names from property sales csv and the Zillow HVI data
## A similarity value threshold of 89 is fixed and the similarity > 89 is selected

In [18]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
match_loc = []
similarity = []
for i in Home_value_zillow_vb.Neighborhood_zillow:
        ratio = process.extract( i, Zip_df.Neighborhood, limit=5)
        match_loc.append(ratio[0][0])
        similarity.append(ratio[0][1])
        ##print(i,similarity,match_loc)
Home_value_zillow_vb['similarity']=similarity
Home_value_zillow_vb['Zip_match']=match_loc
Home_value_zillow_vb

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match
0,246678,Virginia Beach,Princess Anne,369781.0,100,Princess Anne
1,244707,Virginia Beach,Kempsville,256486.0,100,Kempsville
2,247145,Virginia Beach,Salem,299268.0,90,Salem Acre/Meadowridge
3,241514,Virginia Beach,Bayside,253829.0,100,Bayside
4,245508,Virginia Beach,Mears Corner,292956.0,61,Buckner Farms
...,...,...,...,...,...,...
26,417477,Virginia Beach,First Landing State Park,1103866.0,86,Airport Ind Park
27,418158,Virginia Beach,Croatan Beach,870809.0,100,Croatan Beach
28,242226,Virginia Beach,Cape Story by the Sea,696546.0,100,Cape Story By The Sea
29,247090,Virginia Beach,Rudee Heights,380966.0,100,Rudee Heights


## The column Zip_match has the matching neighborhood name from property sales csv which will be used in the next step to map the correct zip code

In [24]:
Home_value_zillow_vb = Home_value_zillow_vb[(Home_value_zillow_vb['similarity'] > 89)]
Home_value_zillow_vb.reset_index(drop=True,inplace=True)
Home_value_zillow_vb

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match
0,246678,Virginia Beach,Princess Anne,369781.0,100,Princess Anne
1,244707,Virginia Beach,Kempsville,256486.0,100,Kempsville
2,247145,Virginia Beach,Salem,299268.0,90,Salem Acre/Meadowridge
3,241514,Virginia Beach,Bayside,253829.0,100,Bayside
4,195088,Virginia Beach,Great Neck,393627.0,95,Great Neck Rd
5,244793,Virginia Beach,Kings Grant,400568.0,90,Eastern Park Kings Grant Cove
6,247231,Virginia Beach,Seatack,320765.0,95,Seatack Sf
7,417478,Virginia Beach,Green Run,227791.0,95,Green Run Sf
8,247895,Virginia Beach,Thalia,258412.0,100,Thalia
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,Princess Anne Plaza


### Using the matched neighborhood name the zip code with maximum occurence is assigned for each of the neighborhood in zillow data

In [34]:
pd.set_option('display.max_rows', 500)
match_loc_zip_neighborhood = []
match_neighborhood = []
match_loc_zip = []
match_loc_zip_count = []
for i in Home_value_zillow_vb.Zip_match:
        ##ratio = process.extract( i, crime_df_summary.Neighborhood, limit=1)
        ##match_loc_crime.append(ratio[0][0])
        ##similarity_crime.append(ratio[0][1])
        occur_count = 0
 ##       print(i)
        for index,row in Zip_df.iterrows():
            id = row['Neighborhood']
            count = row['count']
            Zip = row['Zip Code']
            if re.search(i,id): # this line changed
                match_loc_zip_neighborhood.append (id)
                match_loc_zip_count.append(count)
                match_neighborhood.append(i)
                match_loc_zip.append(Zip)
                ##print(i,count,id)
##match_loc_zip_neighborhood
##match_loc_zip_count
print(len(match_neighborhood),len(match_loc_zip_count),len(match_loc_zip_neighborhood),len(match_loc_zip))
Zip_data = pd.DataFrame({'Neighborhood': match_neighborhood, 'Neighborhood_match': match_loc_zip_neighborhood,'Occur Count':match_loc_zip_count,'Zip Code':match_loc_zip})
Zip_data
##Home_value_zillow_vb['Zip_neighborhood'] = match_loc_zip
##Home_value_zillow_vb['Zip_count'] = match_loc_zip_count
##Home_value_zillow_vb

76 76 76 76


,Neighborhood,Neighborhood_match,Occur Count,Zip Code
0,Princess Anne,Enclave At Princess Anne,67,23456
1,Princess Anne,Lake Princess Anne Condos,625,23456
2,Princess Anne,Princess Anne,2268,23456
3,Princess Anne,Princess Anne Gardens,654,23453
4,Princess Anne,Princess Anne Hills,432,23451
5,Princess Anne,Princess Anne Land Use,147,23456
6,Princess Anne,Princess Anne Meadows,58,23456
7,Princess Anne,Princess Anne Plaza,11038,23452
8,Princess Anne,Princess Anne Plaza Kings Arms,379,23452
9,Princess Anne,Princess Anne Plaza Th/Sec 1,393,23452


### Take the max count and assign one zip code and merge both the dataframes

In [35]:
idx=Zip_data.groupby(['Neighborhood'])['Occur Count'].idxmax()
Zip_df_1=Zip_data.loc[idx,].reset_index()
Zip_df_1['Occur Count'] = Zip_df_1['Occur Count'].astype(int)
Zip_df_1.reset_index(inplace = True)
Zip_df_1

,level_0,index,Neighborhood,Neighborhood_match,Occur Count,Zip Code
0,0,51,Acredale,Acredale,863,23464
1,1,59,Alanton,Alanton,1377,23454
2,2,64,Bay Colony,Bay Colony,2099,23451
3,3,27,Bayside,Bayfront/Bayside Condo Dunes,427,23451
4,4,63,Broad Bay Colony,Broad Bay Colony,674,23451
5,5,69,Cape Story By The Sea,Cape Story By The Sea,1514,23451
6,6,52,Chesapeake Beach,Chesapeake Beach,1846,23455
7,7,67,Croatan Beach,Croatan Beach,760,23451
8,8,33,Eastern Park Kings Grant Cove,Eastern Park Kings Grant Cove,35,23452
9,9,32,Great Neck Rd,Great Neck Rd,109,23454


In [36]:
Home_value_zillow_vb_merged_zip = pd.DataFrame()
Home_value_zillow_vb_merged_zip = pd.merge(Home_value_zillow_vb,Zip_df_1,left_on='Zip_match',right_on='Neighborhood')[['RegionID','City','Neighborhood_zillow','Median_HVI','similarity','Zip Code']]
##Home_value_zillow_vb
Home_value_zillow_vb_merged_zip

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip Code
0,246678,Virginia Beach,Princess Anne,369781.0,100,23452
1,244707,Virginia Beach,Kempsville,256486.0,100,23462
2,247145,Virginia Beach,Salem,299268.0,90,23456
3,241514,Virginia Beach,Bayside,253829.0,100,23451
4,195088,Virginia Beach,Great Neck,393627.0,95,23454
5,244793,Virginia Beach,Kings Grant,400568.0,90,23452
6,247231,Virginia Beach,Seatack,320765.0,95,23451
7,417478,Virginia Beach,Green Run,227791.0,95,23453
8,247895,Virginia Beach,Thalia,258412.0,100,23452
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,23452


## Search the crime DF for Zillow neighborhood name and take cumulative crimecount sum for each neighborhood

In [37]:
match_loc_crime = []
similarity_crime = []
for i in Home_value_zillow_vb.Neighborhood_zillow:
        ##ratio = process.extract( i, crime_df_summary.Neighborhood, limit=1)
        ##match_loc_crime.append(ratio[0][0])
        ##similarity_crime.append(ratio[0][1])
        crime_count = 0
  ##      print(i)
        for index,row in crime_df_summary.iterrows():
            id = row['Neighborhood']
            count = row['Crime Count']
            if re.search(i,id): # this line changed
                crime_count = crime_count+count
 ##               print(i,count,crime_count,id)
        match_loc_crime.append(crime_count)
        ##for index in range(crime_df_summary.shape[1]):          
            ##print('Column Number : ', index) 
        # Select column by index position using iloc[] 
            ##columnSeriesObj = crime_df_summary.iloc[:, index] 
            ##print('Column Contents : ', columnSeriesObj.values) 
##Home_value_zillow_vb['similarity_crime']=similarity_crime
##Home_value_zillow_vb['Zip_match_crime']=match_loc_crime
Home_value_zillow_vb['Crime_count'] = match_loc_crime
Home_value_zillow_vb

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match,Crime_count
0,246678,Virginia Beach,Princess Anne,369781.0,100,Princess Anne,4941
1,244707,Virginia Beach,Kempsville,256486.0,100,Kempsville,4833
2,247145,Virginia Beach,Salem,299268.0,90,Salem Acre/Meadowridge,2631
3,241514,Virginia Beach,Bayside,253829.0,100,Bayside,3736
4,195088,Virginia Beach,Great Neck,393627.0,95,Great Neck Rd,935
5,244793,Virginia Beach,Kings Grant,400568.0,90,Eastern Park Kings Grant Cove,414
6,247231,Virginia Beach,Seatack,320765.0,95,Seatack Sf,1119
7,417478,Virginia Beach,Green Run,227791.0,95,Green Run Sf,4807
8,247895,Virginia Beach,Thalia,258412.0,100,Thalia,2706
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,Princess Anne Plaza,3265


## Wherever search didn't work do a fuzzy compare

In [38]:
Home_value_zillow_vb_cc = Home_value_zillow_vb[(Home_value_zillow_vb['Crime_count'] == 0)]
Home_value_zillow_vb_cc
match_loc_crime = []
similarity_crime = []
for i in Home_value_zillow_vb_cc.Neighborhood_zillow:
        ratio = process.extract( i, crime_df_summary.Neighborhood, limit=1)
        match_loc_crime.append(ratio[0][0])
        similarity_crime.append(ratio[0][1])
Home_value_zillow_vb_cc['match_crime']=match_loc_crime
Home_value_zillow_vb_cc

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match,Crime_count,match_crime
19,242226,Virginia Beach,Cape Story by the Sea,696546.0,100,Cape Story By The Sea,0,Cape Story


In [39]:
Home_value_zillow_vb_cc = pd.merge(Home_value_zillow_vb_cc,crime_df_summary,left_on='match_crime',right_on='Neighborhood')
Home_value_zillow_vb_cc

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match,Crime_count,match_crime,Neighborhood,Crime Count
0,242226,Virginia Beach,Cape Story by the Sea,696546.0,100,Cape Story By The Sea,0,Cape Story,Cape Story,64


In [40]:
Home_value_zillow_vb.loc[Home_value_zillow_vb.Neighborhood_zillow.isin(Home_value_zillow_vb_cc.Neighborhood_zillow), ['Crime_count']] = Home_value_zillow_vb_cc[['Crime Count']].values
Home_value_zillow_vb

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip_match,Crime_count
0,246678,Virginia Beach,Princess Anne,369781.0,100,Princess Anne,4941
1,244707,Virginia Beach,Kempsville,256486.0,100,Kempsville,4833
2,247145,Virginia Beach,Salem,299268.0,90,Salem Acre/Meadowridge,2631
3,241514,Virginia Beach,Bayside,253829.0,100,Bayside,3736
4,195088,Virginia Beach,Great Neck,393627.0,95,Great Neck Rd,935
5,244793,Virginia Beach,Kings Grant,400568.0,90,Eastern Park Kings Grant Cove,414
6,247231,Virginia Beach,Seatack,320765.0,95,Seatack Sf,1119
7,417478,Virginia Beach,Green Run,227791.0,95,Green Run Sf,4807
8,247895,Virginia Beach,Thalia,258412.0,100,Thalia,2706
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,Princess Anne Plaza,3265


## Merge Zip code and crime count data

In [41]:
Home_value_zillow_vb_merged_zip = pd.DataFrame()
Home_value_zillow_vb_merged_zip = pd.merge(Home_value_zillow_vb,Zip_df_1,left_on='Zip_match',right_on='Neighborhood')[['RegionID','City','Neighborhood_zillow','Median_HVI','similarity','Zip Code','Crime_count']]
##Home_value_zillow_vb
Final_df = Home_value_zillow_vb_merged_zip.loc[Home_value_zillow_vb_merged_zip['Zip Code'].notna()]
Final_df.drop(Final_df[Final_df['Zip Code'] == '\t'].index, inplace = True) 
Final_df

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip Code,Crime_count
0,246678,Virginia Beach,Princess Anne,369781.0,100,23452,4941
1,244707,Virginia Beach,Kempsville,256486.0,100,23462,4833
2,247145,Virginia Beach,Salem,299268.0,90,23456,2631
3,241514,Virginia Beach,Bayside,253829.0,100,23451,3736
4,195088,Virginia Beach,Great Neck,393627.0,95,23454,935
5,244793,Virginia Beach,Kings Grant,400568.0,90,23452,414
6,247231,Virginia Beach,Seatack,320765.0,95,23451,1119
7,417478,Virginia Beach,Green Run,227791.0,95,23453,4807
8,247895,Virginia Beach,Thalia,258412.0,100,23452,2706
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,23452,3265


## Importing Libraries for clustering and plotting maps

In [42]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

## Get Latitude and longitude using nominatim zip code query

In [43]:
Zip_lat = []
Zip_long = []
for i in Final_df['Zip Code'].values:
        location = nomi.query_postal_code(i)
        Zip_lat.append(location.latitude)
        Zip_long.append(location.longitude)
        
Final_df['Latitude']=Zip_lat
Final_df['Longitude']=Zip_long
##postal_code = "23464"
Final_df

,RegionID,City,Neighborhood_zillow,Median_HVI,similarity,Zip Code,Crime_count,Latitude,Longitude
0,246678,Virginia Beach,Princess Anne,369781.0,100,23452,4941,36.8348,-76.0961
1,244707,Virginia Beach,Kempsville,256486.0,100,23462,4833,36.8392,-76.1522
2,247145,Virginia Beach,Salem,299268.0,90,23456,2631,36.7341,-76.0359
3,241514,Virginia Beach,Bayside,253829.0,100,23451,3736,36.8585,-76.0019
4,195088,Virginia Beach,Great Neck,393627.0,95,23454,935,36.8282,-76.0237
5,244793,Virginia Beach,Kings Grant,400568.0,90,23452,414,36.8348,-76.0961
6,247231,Virginia Beach,Seatack,320765.0,95,23451,1119,36.8585,-76.0019
7,417478,Virginia Beach,Green Run,227791.0,95,23453,4807,36.7760,-76.0766
8,247895,Virginia Beach,Thalia,258412.0,100,23452,2706,36.8348,-76.0961
9,246679,Virginia Beach,Princess Anne Plaza,245260.0,100,23452,3265,36.8348,-76.0961


## Get Foursquare ID and credentials

In [44]:
CLIENT_ID = 'BO2RSPOKG4T2FDWBUGMV120DBWMQZZ0NA5BIX4DQRKFWNIFX' # your Foursquare ID
CLIENT_SECRET = 'DHKBT11KL13FQMLREDDEBTDA3SO10POGHS44BVE2NOJYEUSF' # your Foursquare Secret
VERSION = '20200604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BO2RSPOKG4T2FDWBUGMV120DBWMQZZ0NA5BIX4DQRKFWNIFX
CLIENT_SECRET:DHKBT11KL13FQMLREDDEBTDA3SO10POGHS44BVE2NOJYEUSF


## Define Function get venue data from Foursquare using latitude and longitude

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        ##print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            category)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Call Foursquare for venue data with a 2500 radius

In [46]:
LIMIT = 100
category=''
VB_venues = getNearbyVenues(names=Final_df['Neighborhood_zillow'],
                                   latitudes=Final_df['Latitude'],
                                   longitudes=Final_df['Longitude'],
                                   radius=2500
                                  )

## Call Foursquare for school data with a 5000 radius

In [47]:
LIMIT = 100
category='4f4533804b9074f6e4fb0105,4bf58dd8d48988d13d941735,4f4533814b9074f6e4fb0106,52e81612bcbc57f1066b7a46'
VB_schools = getNearbyVenues(names=Final_df['Neighborhood_zillow'],
                                   latitudes=Final_df['Latitude'],
                                   longitudes=Final_df['Longitude'],
                                   radius=5000
                                  )

## Call Foursquare for hospital data with a 2500 radius

In [48]:
category='4bf58dd8d48988d196941735,56aa371be4b08b9a8d573526'
VB_hospitals = getNearbyVenues(names=Final_df['Neighborhood_zillow'],
                                   latitudes=Final_df['Latitude'],
                                   longitudes=Final_df['Longitude'],
                                   radius=10000
                                  )

## Do one hot encoding for venue data and group by neighborhood

In [49]:
VB_onehot = pd.get_dummies(VB_venues[['Venue Category']], prefix="", prefix_sep="")
VB_onehot['Neighborhood'] = VB_venues['Neighborhood']
##VB_onehot

In [50]:
VB_grouped = VB_onehot.groupby('Neighborhood').mean().reset_index()
##VB_grouped

## Define function to return top 10 venues per neighborhood

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
VB_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
VB_neighborhoods_venues_sorted['Neighborhood'] = VB_grouped['Neighborhood']

for ind in np.arange(VB_grouped.shape[0]):
    VB_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(VB_grouped.iloc[ind, :], num_top_venues)

VB_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acredale,Sandwich Place,Fast Food Restaurant,Pizza Place,Convenience Store,Bakery,Coffee Shop,Mexican Restaurant,Sushi Restaurant,Supermarket,American Restaurant
1,Alanton,Discount Store,Cosmetics Shop,Hotel,Seafood Restaurant,Frozen Yogurt Shop,Sports Bar,Drugstore,Big Box Store,Massage Studio,Mexican Restaurant
2,Bay Colony,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
3,Bayside,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
4,Broad Bay Colony,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant


## Do one hot encoding for both hospitals and schools

In [53]:
VB_onehot_schools = pd.get_dummies(VB_schools[['Venue Category']], prefix="", prefix_sep="")
VB_onehot_schools['Neighborhood'] = VB_schools['Neighborhood']
VB_onehot_hospitals = pd.get_dummies(VB_hospitals[['Venue Category']], prefix="", prefix_sep="")
VB_onehot_hospitals['Neighborhood'] = VB_hospitals['Neighborhood']
##VB_onehot_schools

In [54]:
VB_schools_grouped = VB_onehot_schools.groupby('Neighborhood').mean().reset_index()
VB_hospitals_grouped = VB_onehot_hospitals.groupby('Neighborhood').mean().reset_index()
##VB_schools_grouped=pd.qcut(VB_schools_grouped['Daycare'], bins=[0, 0.4, 0.6, 0.8], include_lowest=True, labels=['low', 'mid', 'high'])
##VB_schools_grouped

## Merge School and healtcare data

In [55]:
VB_grouped_Ed_HC_final = []
VB_grouped_Ed_HC_final = VB_schools_grouped.merge(VB_hospitals_grouped, on='Neighborhood', how='left', indicator=True)
VB_grouped_Ed_HC_final

,Neighborhood,Daycare,Elementary School,High School,Middle School,Private School,Doctor's Office,Home Service,Hospital,Hospital Ward,Medical Center,Medical Lab,Urgent Care Center,_merge
0,Acredale,0.035714,0.571429,0.178571,0.178571,0.035714,0.024390,0.024390,0.731707,0.024390,0.073171,0.000000,0.121951,both
1,Alanton,0.062500,0.687500,0.000000,0.250000,0.000000,0.000000,0.000000,0.821429,0.000000,0.035714,0.035714,0.107143,both
2,Bay Colony,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,both
3,Bayside,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,both
4,Broad Bay Colony,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,both
5,Cape Story by the Sea,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,both
6,Chesapeake Beach,0.000000,0.684211,0.105263,0.105263,0.105263,0.038462,0.038462,0.576923,0.038462,0.192308,0.000000,0.115385,both
7,Croatan Beach,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,both
8,Great Neck,0.062500,0.687500,0.000000,0.250000,0.000000,0.000000,0.000000,0.821429,0.000000,0.035714,0.035714,0.107143,both
9,Green Run,0.000000,0.722222,0.166667,0.111111,0.000000,0.000000,0.000000,0.680000,0.000000,0.120000,0.040000,0.160000,both


In [56]:
VB_grouped_HVI_Safety_binned=[]
selector_d = {'Neighborhood_zillow': 'Neighborhood', 'Median_HVI': 'Home_Value_Index','Crime_count': 'Crime_count','Zip Code':'Zip_Code','Latitude':'Latitude','Longitude':'Longitude'}
VB_grouped_HVI_Safety = Final_df.rename(columns=selector_d)[[*selector_d.values()]]
VB_grouped_HVI_Safety_binned = VB_grouped_HVI_Safety

## Merge HVI, safety, school and hospital data

In [57]:
VB_grouped_Ed_HC_HVI_Safety = VB_grouped_Ed_HC_final.merge(VB_grouped_HVI_Safety, on='Neighborhood', how='left', indicator='exists')
VB_grouped_Ed_HC_HVI_Safety.drop('_merge', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety.drop('exists', 1,inplace=True)
##VB_grouped_Ed_HC_HVI_Safety.drop('_merge', 1,inplace=True)
##VB_grouped_Ed_HC_HVI_Safety.drop('exists', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety_Venue = VB_grouped_Ed_HC_HVI_Safety.merge(VB_grouped, on='Neighborhood', how='left', indicator='exists')
VB_grouped_Ed_HC_HVI_Safety_Venue.drop('exists', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety_Venue['Home_Value_Index'] = VB_grouped_Ed_HC_HVI_Safety_Venue['Home_Value_Index']/1000000
VB_grouped_Ed_HC_HVI_Safety_Venue['Crime_count'] = VB_grouped_Ed_HC_HVI_Safety_Venue['Crime_count']/1000
##VB_grouped_Ed_HC_HVI_Safety_Venue

## Set the K value from 5 to 9 incrementing by 1 and running the algorithm. 8 was found to be best value

In [78]:
# set number of clusters
kclusters = 8
VB_grouped_clustering = VB_grouped_Ed_HC_HVI_Safety_Venue.drop(['Neighborhood', 'Zip_Code', 
                'Latitude', 'Longitude'], 1)
VB_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(VB_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
##VB_grouped_Ed_HC_HVI_Safety

array([3, 3, 1, 5, 1, 1, 3, 1, 6, 4, 4, 3, 0, 2, 4, 5, 3, 0, 7, 6, 0, 3],
      dtype=int32)

In [81]:
if 'Cluster Labels' in VB_grouped_Ed_HC_HVI_Safety.columns:
    VB_grouped_Ed_HC_HVI_Safety_Venue.drop('Cluster Labels', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety_Venue.insert(0, 'Cluster Labels', kmeans.labels_)
##VB_grouped_Ed_HC_HVI_Safety_Venue

In [86]:
VB_grouped_Ed_HC_HVI_Safety_Venue.loc[VB_grouped_Ed_HC_HVI_Safety_Venue['Cluster Labels'] == 0, VB_grouped_Ed_HC_HVI_Safety_Venue.columns[[0] + list(range(5, VB_grouped_Ed_HC_HVI_Safety_Venue.shape[0]))]]

,Cluster Labels,Middle School,Private School,Doctor's Office,Home Service_x,Hospital,Hospital Ward,Medical Center,Medical Lab,Urgent Care Center,Home_Value_Index,Crime_count,Zip_Code,Latitude,Longitude,ATM,American Restaurant,Antique Shop
12,0,0.250000,0.0,0.00000,0.00000,0.821429,0.00000,0.035714,0.035714,0.107143,0.231062,2.686,23454,36.8282,-76.0237,0.00,0.000000,0.00
17,0,0.181818,0.0,0.00000,0.00000,0.785714,0.00000,0.000000,0.071429,0.142857,0.299268,2.631,23456,36.7341,-76.0359,0.00,0.047619,0.00
20,0,0.076923,0.0,0.02381,0.02381,0.738095,0.02381,0.095238,0.000000,0.095238,0.258412,2.706,23452,36.8348,-76.0961,0.01,0.040000,0.01


In [95]:
if 'Cluster Labels' in VB_grouped_Ed_HC_HVI_Safety.columns:
    VB_grouped_Ed_HC_HVI_Safety.drop('Cluster Labels', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety.insert(0, 'Cluster Labels', kmeans.labels_)
VB_grouped_Ed_HC_HVI_Safety_top10 = VB_grouped_Ed_HC_HVI_Safety.merge(VB_neighborhoods_venues_sorted, on='Neighborhood', how='left', indicator='exists')
VB_grouped_Ed_HC_HVI_Safety_top10.drop('exists', 1,inplace=True)
VB_grouped_Ed_HC_HVI_Safety_top10

,Cluster Labels,Neighborhood,Daycare,Elementary School,High School,Middle School,Private School,Doctor's Office,Home Service,Hospital,Hospital Ward,Medical Center,Medical Lab,Urgent Care Center,Home_Value_Index,Crime_count,Zip_Code,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Acredale,0.035714,0.571429,0.178571,0.178571,0.035714,0.024390,0.024390,0.731707,0.024390,0.073171,0.000000,0.121951,332126.0,320,23464,36.7978,-76.1759,Sandwich Place,Fast Food Restaurant,Pizza Place,Convenience Store,Bakery,Coffee Shop,Mexican Restaurant,Sushi Restaurant,Supermarket,American Restaurant
1,3,Alanton,0.062500,0.687500,0.000000,0.250000,0.000000,0.000000,0.000000,0.821429,0.000000,0.035714,0.035714,0.107143,495249.0,116,23454,36.8282,-76.0237,Discount Store,Cosmetics Shop,Hotel,Seafood Restaurant,Frozen Yogurt Shop,Sports Bar,Drugstore,Big Box Store,Massage Studio,Mexican Restaurant
2,1,Bay Colony,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,892192.0,158,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
3,5,Bayside,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,253829.0,3736,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
4,1,Broad Bay Colony,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,840671.0,44,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
5,1,Cape Story by the Sea,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,696546.0,64,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
6,3,Chesapeake Beach,0.000000,0.684211,0.105263,0.105263,0.105263,0.038462,0.038462,0.576923,0.038462,0.192308,0.000000,0.115385,443098.0,542,23455,36.8881,-76.1446,Pizza Place,Discount Store,Coffee Shop,Mexican Restaurant,Supermarket,Smoothie Shop,Department Store,Fast Food Restaurant,Grocery Store,Sandwich Place
7,1,Croatan Beach,0.000000,0.666667,0.166667,0.166667,0.000000,0.000000,0.000000,0.750000,0.000000,0.062500,0.000000,0.187500,870809.0,79,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
8,6,Great Neck,0.062500,0.687500,0.000000,0.250000,0.000000,0.000000,0.000000,0.821429,0.000000,0.035714,0.035714,0.107143,393627.0,935,23454,36.8282,-76.0237,Discount Store,Cosmetics Shop,Hotel,Seafood Restaurant,Frozen Yogurt Shop,Sports Bar,Drugstore,Big Box Store,Massage Studio,Mexican Restaurant
9,4,Green Run,0.000000,0.722222,0.166667,0.111111,0.000000,0.000000,0.000000,0.680000,0.000000,0.120000,0.040000,0.160000,227791.0,4807,23453,36.7760,-76.0766,Gym / Fitness Center,Furniture / Home Store,Mexican Restaurant,Discount Store,Clothing Store,Sandwich Place,Martial Arts School,Bar,Cosmetics Shop,Mobile Phone Shop


In [94]:
VB_grouped_Ed_HC_HVI_Safety_top10.loc[VB_grouped_Ed_HC_HVI_Safety_top10['Cluster Labels'] == 1, VB_grouped_Ed_HC_HVI_Safety_top10.columns[[0] + list(range(1, VB_grouped_Ed_HC_HVI_Safety_top10.shape[1]))]]

,Cluster Labels,Neighborhood,Daycare,Elementary School,High School,Middle School,Private School,Doctor's Office,Home Service,Hospital,Hospital Ward,Medical Center,Medical Lab,Urgent Care Center,Home_Value_Index,Crime_count,Zip_Code,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,1,Bay Colony,0.0,0.666667,0.166667,0.166667,0.0,0.0,0.0,0.75,0.0,0.0625,0.0,0.1875,892192.0,158,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
4,1,Broad Bay Colony,0.0,0.666667,0.166667,0.166667,0.0,0.0,0.0,0.75,0.0,0.0625,0.0,0.1875,840671.0,44,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
5,1,Cape Story by the Sea,0.0,0.666667,0.166667,0.166667,0.0,0.0,0.0,0.75,0.0,0.0625,0.0,0.1875,696546.0,64,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant
7,1,Croatan Beach,0.0,0.666667,0.166667,0.166667,0.0,0.0,0.0,0.75,0.0,0.0625,0.0,0.1875,870809.0,79,23451,36.8585,-76.0019,Seafood Restaurant,Coffee Shop,Grocery Store,Beach,Sandwich Place,Hotel,American Restaurant,Italian Restaurant,Pizza Place,Mediterranean Restaurant


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(VB_grouped_Ed_HC_HVI_Safety_top10['Latitude'], VB_grouped_Ed_HC_HVI_Safety_top10['Longitude'], VB_grouped_Ed_HC_HVI_Safety_top10['Neighborhood'], VB_grouped_Ed_HC_HVI_Safety_top10['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters